In [11]:
from spotipy import Spotify
from spotipy.oauth2 import SpotifyOAuth, SpotifyClientCredentials
from spotipy.exceptions import SpotifyException
import time
import pandas as pd

# Replace these with your own client ID and client secret
CLIENT_ID = '4788c668b1514918954e31dfd7bd366d'
CLIENT_SECRET = 'de2a40bde1d84fb9a04ae6a4d71f50cb'
REDIRECT_URI = 'http://localhost:8888/callback'  

# Define the required scopes (permissions)
SCOPE = 'user-library-read, playlist-read-private'

sp_oauth = SpotifyOAuth(client_id=CLIENT_ID, client_secret=CLIENT_SECRET, redirect_uri=REDIRECT_URI, scope=SCOPE, show_dialog=True)

# Authentication
sp = Spotify(auth_manager=sp_oauth)

In [12]:
# List to store track information
tracks_data = []

# Pagination variables
offset = 0
limit = 100  # Maximum number of tracks per request
playlist_id = '23QFKmMd3B7LAcRNWEX6vM'

while True:
    # Fetch tracks from the playlist
    results = sp.playlist_tracks(playlist_id, limit=limit, offset=offset)
    
    # Process each track in the playlist
    for item in results['items']:
        track = item['track']
        if track:
            track_info = {
                'name': track['name'],
                'artist': track['artists'][0]['name'],
                'album': track['album']['name'],
                'popularity': track['popularity']
            }
            tracks_data.append(track_info)
    
    # Break the loop if all tracks are fetched
    if len(results['items']) < limit:
        break
    
    # Increment the offset to fetch the next set of tracks
    offset += limit

# Create a DataFrame from the tracks data
df = pd.DataFrame(tracks_data)

In [13]:
df

,name,artist,album,popularity
0,Goodie Bag,Still Woozy,Goodie Bag,74
1,BS,Still Woozy,BS,52
2,Lucy,Still Woozy,Lucy,62
3,I Wanna Be Yours,Arctic Monkeys,AM,91
4,Sex on Fire,Kings of Leon,Only By The Night,85
...,...,...,...,...
124,Brandy (You're a Fine Girl),Looking Glass,Looking Glass,78
125,The Middle,Jimmy Eat World,Bleed American,82
126,Love Grows (Where My Rosemary Goes),Edison Lighthouse,Love Grows (Where My Rosemary Goes) & Other Gems,72
127,Come Together - Remastered 2015,The Beatles,1 (Remastered),48


In [14]:
playlist_id = '23QFKmMd3B7LAcRNWEX6vM'
limit_per_request = 100

# List to store audio features
audio_features_data = []

# Pagination variables
offset = 0

while True:
    # Fetch tracks from the playlist
    results = sp.playlist_tracks(playlist_id, limit=limit_per_request, offset=offset)

    # Collect track IDs
    track_ids = [item['track']['id'] for item in results['items'] if item['track']]

    # Get audio features for each track in chunks of 100
    for i in range(0, len(track_ids), 100):
        features_list = sp.audio_features(track_ids[i:i + 100])
        for features in features_list:
            if features:
                audio_features_info = {
                    'id': features['id'],
                    'danceability': features['danceability'],
                    'energy': features['energy'],
                    'key': features['key'],
                    'loudness': features['loudness'],
                    'mode': features['mode'],
                    'speechiness': features['speechiness'],
                    'acousticness': features['acousticness'],
                    'instrumentalness': features['instrumentalness'],
                    'liveness': features['liveness'],
                    'valence': features['valence'],
                    'tempo': features['tempo'],
                    'time_signature': features['time_signature']
                }
                audio_features_data.append(audio_features_info)

    # Break the loop if all tracks are fetched
    if len(results['items']) < limit_per_request:
        break

    # Increment the offset to fetch the next set of tracks
    offset += limit_per_request

# Create a DataFrame from the audio features data
df_audio_features = pd.DataFrame(audio_features_data)

Max Retries reached


SpotifyException: http status: 429, code:-1 - /v1/audio-features/?ids=4vHNeBWDQpVCmGbaccrRzi,1qxZswAdBvM0JMUoV83J1b,5RxpYHVbGJPOvSEATQyg9P,5XeFesFbtLpXzIVDNQP22n,0ntQJM78wzOLVeCUAW7Y45,0NdTUS4UiNYCNn5FgVqKQY,086myS9r57YsLbJpU0TgK9,58ge6dfP91o9oXMzq3XkIS,5VGlqQANWDKJFl0MBG3sg2,6ubPFMzlIg73d1jzzWcC8B,2tznHmp70DxMyr2XhWLOW0,4sebUbjqbcgDSwG6PbSGI0,6K4t31amVTZDgR3sKmwUJJ,0LtOwyZoSNZKJWHqjzADpW,7nzsY8vlnKdvGOEE0rjAXZ,5n0CTysih20NYdT2S0Wpe8,2AYEOC02WLhUiOoaig2SEH,0GO8y8jQk1PkHzS31d699N,2AT8iROs4FQueDv2c8q2KE,1170VohRSx6GwE6QDCHPPH,7yNf9YjeO5JXUE3JEBgnYc,0UV5zxRMz6AO4ZwUOZNIKI,3n69hLUdIsSa1WlRmjMZlW,1XrSjpNe49IiygZfzb74pk,5hM5arv9KDbCHS0k9uqwjr,1Q3t9fWvHUXKsMmpD2XpUu,5FVd6KXrgO9B3JPmC8OPst,0awZwmlC6pxH65KTZpadmX,4iEOVEULZRvmzYSZY2ViKN,74f0dpqHhTeTJzs4pmZ1yq,4MXE6VCvTsQitHWrAxj7Kg,2INUkDPCO1eQlOFmIN5VAC,5kBYMymc2d3h164QqEHwRL,3S5mohVxC0Xuj0tgZ7vU7g,2UAWVtEwFLJqdJsT8FCzKX,0tkBOcK7oRVXQJY97zzSvr,6OiRh4kttAs1YWglvTcYkB,6qZjm61s6u8Ead9sWxCDro,57Xjny5yNzAcsxnusKmAfA,7h2nmmoWDi2UpfYKLKWLYB,3rPtS4nfpy

In [ ]:
playlist_id = '23QFKmMd3B7LAcRNWEX6vM'
limit_per_request = 100
delay = 1

# List to store audio analysis data
audio_analysis_data = []

# Pagination variables
offset = 0

while True:
    # Fetch tracks from the playlist
    results = sp.playlist_tracks(playlist_id, limit=limit_per_request, offset=offset)

    # Collect track IDs
    track_ids = [item['track']['id'] for item in results['items'] if item['track']]

    # Get audio analysis for each track
    for track_id in track_ids:
        analysis = sp.audio_analysis(track_id)
        if analysis:
            audio_analysis_info = {
                'id': track_id,
                'duration': analysis['track']['duration'],
                'tempo_analysis': analysis['track']['tempo'],
                'time_signature_analysis': analysis['track']['time_signature'],
                'key_analysis': analysis['track']['key'],
                'mode_analysis': analysis['track']['mode'],
                'loudness_analysis': analysis['track']['loudness'],
                'sections': len(analysis['sections']),
                'segments': len(analysis['segments']),
                'beats': len(analysis['beats']),
                'bars': len(analysis['bars']),
                'tatums': len(analysis['tatums'])
            }
            audio_analysis_data.append(audio_analysis_info)

    # Break the loop if all tracks are fetched
    if len(results['items']) < limit_per_request:
        break

    # Increment the offset to fetch the next set of tracks
    offset += limit_per_request

# Create a DataFrame from the audio analysis data
df_audio_analysis = pd.DataFrame(audio_analysis_data)

In [17]:
sp.audio_features?

Signature: sp.audio_features(tracks=[])
Docstring:
Get audio features for one or multiple tracks based upon their Spotify IDs
Parameters:
    - tracks - a list of track URIs, URLs or IDs, maximum: 100 ids
File:      c:\users\shank\anaconda3\envs\dsenv\lib\site-packages\spotipy\client.py
Type:      method

In [18]:
sp.playlist_tracks?

Signature:
sp.playlist_tracks(
    playlist_id,
    fields=None,
    limit=100,
    offset=0,
    market=None,
    additional_types=('track',),
)
Docstring:
Get full details of the tracks of a playlist.

Parameters:
    - playlist_id - the playlist ID, URI or URL
    - fields - which fields to return
    - limit - the maximum number of tracks to return
    - offset - the index of the first track to return
    - market - an ISO 3166-1 alpha-2 country code.
    - additional_types - list of item types to return.
                         valid types are: track and episode
File:      c:\users\shank\anaconda3\envs\dsenv\lib\site-packages\spotipy\client.py
Type:      method

In [19]:
sp.audio_analysis?

Signature: sp.audio_analysis(track_id)
Docstring:
Get audio analysis for a track based upon its Spotify ID
Parameters:
    - track_id - a track URI, URL or ID
File:      c:\users\shank\anaconda3\envs\dsenv\lib\site-packages\spotipy\client.py
Type:      method

In [24]:
sp.audio_features(['5XeFesFbtLpXzIVDNQP22n', '58ge6dfP91o9oXMzq3XkIS'])

Max Retries reached


SpotifyException: http status: 429, code:-1 - /v1/audio-features/?ids=5XeFesFbtLpXzIVDNQP22n,58ge6dfP91o9oXMzq3XkIS:
 Max Retries, reason: too many 429 error responses

In [25]:
sp.tracks(['5XeFesFbtLpXzIVDNQP22n','58ge6dfP91o9oXMzq3XkIS'])

{'tracks': [{'album': {'album_type': 'album',
    'artists': [{'external_urls': {'spotify': 'https://open.spotify.com/artist/7Ln80lUS6He07XvHI8qqHH'},
      'href': 'https://api.spotify.com/v1/artists/7Ln80lUS6He07XvHI8qqHH',
      'id': '7Ln80lUS6He07XvHI8qqHH',
      'name': 'Arctic Monkeys',
      'type': 'artist',
      'uri': 'spotify:artist:7Ln80lUS6He07XvHI8qqHH'}],
    'available_markets': ['AR',
     'AU',
     'AT',
     'BE',
     'BO',
     'BR',
     'BG',
     'CA',
     'CL',
     'CO',
     'CR',
     'CY',
     'CZ',
     'DK',
     'DO',
     'DE',
     'EC',
     'EE',
     'SV',
     'FI',
     'FR',
     'GR',
     'GT',
     'HN',
     'HK',
     'HU',
     'IS',
     'IE',
     'IT',
     'LV',
     'LT',
     'LU',
     'MY',
     'MT',
     'MX',
     'NL',
     'NZ',
     'NI',
     'NO',
     'PA',
     'PY',
     'PE',
     'PH',
     'PL',
     'PT',
     'SG',
     'SK',
     'ES',
     'SE',
     'CH',
     'TW',
     'TR',
     'UY',
     'US',
     'GB'